判断一手扑克牌(手牌数为5)是否为顺子，假设大小王初始值为0，并可以转换成任意牌面值。

In [15]:
# 思路：易得当手牌中存在对子时就不可能是顺子；
# 然后满足顺子的必要条件为max-min<5，没有抽到大小王时，max-min=4；抽到时则小于4
# 所以大小王需要特殊对待。设立一个记录出现次数的数字，然后在维护一个最大最小值即可


def IsContinuous(cards):
    if not cards or len(cards) != 5:
        return False

    cnts = [0]*14
    cnts[0] = -1    # 代码中遇到>1的值会返回False，而大小王只有两张牌
    min_val, max_val = 14, 0
 
    for val in cards:
        cnts[val] += 1

        if val == 0:
            continue

        if cnts[val] > 1:
            return False

        if val > max_val:
            max_val = val
        if val < min_val:
            min_val = val

    return True if max_val-min_val < 5 else False

Maximum Average Subarray With Length K. 给定一个数组与窗口大小k，求窗口可能的最大(平均)值。

In [18]:
# 思路：扫描一遍数组，使用额外变量记录最大和与出现最大和的位置即可


def findMaxAverage(nums: 'List[int]', k: int) -> float:
    max_sum = sum([nums[i] for i in range(k)])
    cur_sum = max_sum

    for idx in range(1, len(nums)-k+1):
        cur_sum = cur_sum-nums[idx-1]+nums[idx+k-1]
        if cur_sum > max_sum:
            max_sum = cur_sum
            pos = idx

    return max_sum/k

Robot Return to Origin. 二维平面上有一机器人，给定一个字符串形式的移动序列指令，判断机器人是否回到了原点。

In [22]:
def judgeCircle(moves):
    d = {
        'U': 0,
        'D': 0,
        'L': 0,
        'R': 0
    }

    for move in moves:
        d[move] += 1
    return d['U'] == d['D'] and d['L'] == d['R']

Intersection of Two Arrays II. 查找两数组的交集部分。(不是求公共子序列，只是求子集)

In [23]:
# 思路：1. 排序后使用双指针，双指针指向值相等时就加入返回列表，继续扫描
# 2. 数字计数，将长度较短的数组所有数字都记录下来


def intersect(nums1, nums2):
    cnt = dict()
    for num in nums1:
        if num in cnt:
            cnt[num] += 1
        else:
            cnt[num] = 1

    res = []
    for num in nums2:
        if num in cnt and cnt[num] > 0:
            res.append(num)
            cnt[num] -= 1

    return res

Available Captures for Rook. 棋盘上车能吃的棋子数。8*8的国际棋盘，有一个白车(R)，若干白象(B)和黑卒(p)，求白车当前捕获到的对方旗子数量。

In [24]:
# 思路：上下左右四个方位必须直线相通才能捕获到，并且不能被B阻拦
# 输入字符串化，在行内判断是否存在'pR'或'Rp'
# 在列的方向上判断是否存在'pR'或'Rp'


def numRookCaptures(board):
    res = 0
    # 首先索引到R的坐标
    for row in board:
        if 'R' in row:
            tmp = ''.join(row).replace('.','')
            res += 1 if 'pR' in tmp else 0
            res += 1 if 'Rp' in tmp else 0

            col_idx = row.index('R')
            break

    # 将'R'所在的一列提取出来
    col = [row[col_idx] for row in board]
    col = ''.join(col).replace('.','')
    res += 1 if 'pR' in col else 0
    res += 1 if 'Rp' in col else 0

    return res

[Fizz Buzz](https://leetcode.com/problems/fizz-buzz/). 自然数数组，替换其中的数字，替换规则如下：若能被3整除，用'Fizz'替换，若能被5整除，用'Buzz'替换，两者都满足，用'FizzBuzz'替换。

In [28]:
# 思路：把替换规则用哈希表存起来


def fizzBuzz(n: int):
    replace_table = {
        3: 'Fizz',
        5: 'Buzz',
    }
    ans = []

    for num in range(1, n+1):
        re_str = ''
        for key in replace_table:
            if num % key == 0:
                re_str += replace_table[key]

        ans.append(re_str) if re_str else ans.append(str(num))

    return ans

[Game of Life](https://leetcode.com/problems/game-of-life/)。给定一个细胞矩阵，0代表死细胞，1代表活细胞。细胞的状态转移规则如下：
- 周围活细胞数目为2或3则该细胞正常存活
- 周围活细胞数目小于2则该细胞死亡
- 周围活细胞数目大于3则该细胞死亡
- 周围活细胞数目等于3则该细胞复活

In [3]:
# 思路：程序实现时是逐行逐列扫描并改变矩阵值的，那么怎么保存转换之前的转态是关键
# 使用状态码，不难得出有四种可能的状态：
# 0.死细胞变死细胞；1.活细胞变死细胞；2.死细胞变活细胞；3.活细胞变活细胞
# 注意这里状态码的设置有一个技巧，状态码模2为转换前的存活状态，状态码整除2为转换后的存活状态
# 并且模2运算的状态与初始状态相同
# 扫描矩阵两次，第一次做模运算设置状态码，第二次做整除运算设置存活状态


def gameOfLife(board) -> None:
    rows, cols = len(board), len(board[0])
    delta = list(zip([-1, -1, -1, 0, 1, 1, 1, 0], [-1, 0, 1, 1, 1, 0, -1, -1]))

    def check_staus(row, col):
        live_nb = 0    # 转换前存活的邻居数

        for dx, dy in delta:
            x = row + dx
            y = col + dy
            if 0 <= x < rows and 0 <= y < cols and board[x][y] % 2 == 1:
                live_nb += 1

        if board[row][col] % 2 == 0 and live_nb == 3:    # 死细胞复活
            return 2
        elif board[row][col] % 2 == 1:    # 之前为活细胞
            if live_nb == 2 or live_nb == 3:    # 存活
                return 3
            else:    # 死亡
                return 1
        else:
            return 0

    # 设置状态码
    for row in range(rows):
        for col in range(cols):
            board[row][col] = check_staus(row, col)

    # 根据状态码设置存活状态
    for row in range(rows):
        for col in range(cols):
            board[row][col] = board[row][col]//2

[[0, 0, 0], [1, 0, 1], [0, 1, 1], [0, 1, 0]]